<a href="https://www.kaggle.com/code/kushalpatel1124/uber-dataset?scriptVersionId=143937028" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%pip install -q ipywidgets
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, widgets , Layout, Box, Output, Dropdown, interactive
import math
from datetime import datetime

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
file = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        file.append(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
kp = pd.read_csv(file[0])
df = kp.copy()
df

In [ ]:
def get_distance_from_lat_lon_in_km(lat1, lon1, lat2, lon2):
    R = 6371 
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat / 2) * math.sin(dLat / 2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon / 2) * math.sin(dLon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c
    return d
pickup_longitude = -73.999817
pickup_latitude = 40.738354
dropoff_longitude = -73.999512
dropoff_latitude = 40.723217
distance = get_distance_from_lat_lon_in_km(pickup_latitude, pickup_longitude, dropoff_latitude, dropoff_longitude)

In [ ]:
df['distance'] = np.zeros(df.shape[0])
df['cost_per_km'] = np.zeros(df.shape[0])

In [ ]:
for index in df.index:
    df.at[index,'distance'] = round(get_distance_from_lat_lon_in_km(df.at[index,'pickup_latitude'],df.at[index,'pickup_longitude'],df.at[index,'dropoff_latitude'],df.at[index,'dropoff_longitude']),2)

In [ ]:
exchange_rate = 75
df['in_rupees'] = df['fare_amount'] * exchange_rate

In [ ]:
for index in df.index:
    if df.at[index, 'distance'] == 0:
        df.at[index, 'distance'] = df.at[index, 'cost_per_km']
    else:
        df.at[index, 'cost_per_km'] = df.at[index, 'fare_amount'] / df.at[index, 'distance']

In [ ]:
df['cost_per_km'] = round(df['cost_per_km'], 2)

In [ ]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['year'] = df['pickup_datetime'].dt.year
df['month'] = df['pickup_datetime'].dt.month

In [ ]:
df

In [ ]:
passenger_count = df.groupby('year')[['passenger_count' , 'fare_amount']].sum()
passenger_count

In [ ]:
yearly_count = df.groupby('year')['passenger_count'].sum()
plt.figure(figsize=(10, 5))
plt.bar(yearly_count.index, yearly_count.values, color='blue', label='Bar Plot')
plt.plot(yearly_count.index, yearly_count.values, linestyle='-' , color='red')
plt.xlabel('Year')
plt.ylabel('Total Passenger Count')
plt.title('Yearly Passenger Count')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
passenger_count['ave_fare_amount'] = df.groupby('year')['fare_amount'].transform('mean')
passenger_count['percentage_amount'] = (passenger_count['fare_amount'] / passenger_count['fare_amount'].sum()) * 100
passenger_count['percentage_amount'] = round(passenger_count['percentage_amount'], 2)
passenger_count['distance_cover'] = df.groupby('year')['distance'].sum()
grouped = passenger_count.reset_index()
grouped

In [ ]:
passenger_count['previous_year_percentage'] = passenger_count['percentage_amount'].shift(1)
passenger_count['percentage_difference'] = passenger_count['percentage_amount'] - passenger_count['previous_year_percentage']
passenger_count

In [ ]:
year_percentage = passenger_count.groupby('year')['percentage_amount'].sum()
plt.figure(figsize=(5, 5))
plt.pie(year_percentage, labels=year_percentage.index, autopct='%1.1f%%')
plt.title('Yearly Percentage Amount Distribution')
plt.axis('equal')
plt.show()

In [ ]:
def graph(Year):
    if Year:
        yearss = df[(df['year'] == Year)]
        passengerss = yearss.groupby('month')['passenger_count'].sum()
    plt.figure(figsize=(10,5))
    plt.plot(passengerss.index, passengerss.values, linestyle = '-', color = 'red')
    plt.xlabel('Months')
    plt.ylabel('total_passenger_count')
    plt.title('monthly passenger count')
    plt.grid(True)
    plt.show()
uniq_year = df['year'].unique()
uniq_year.sort()
interactive(
    graph,
    Year=widgets.Dropdown(
    options=uniq_year,
    value=2009,
    description="Years:"),
)

In [ ]:
def create_graph(Year='', Month=''):
    filtered_df = df.copy()
    if Year:
        filtered_df = filtered_df[(filtered_df['year'] == Year)]
    if Month != 'All':
        filtered_df = filtered_df[(filtered_df['month'] == Month)]
    
    revenue = filtered_df['fare_amount'].sum()
    revenue = round(revenue, 2)
    print(revenue)

uniq_year = df['year'].unique()
uniq_year.sort()

uni_month = df['month'].unique()
uni_month.sort()

options_month = [('All', 'All')] + [(month, month) for month in uni_month]

interactive(
    create_graph,
    Year=widgets.Dropdown(
        options=uniq_year,
        value=2009,
        description='Years:'
    ),
    
    Month=widgets.Dropdown(
        options=options_month,
        description="Months:"
    )
)

In [ ]:
df2 = pd.DataFrame(df, columns=['year', 'passenger_count', 'month'])
pivot_table = df.pivot_table(index='year', columns='month', values='passenger_count', aggfunc='sum')

In [ ]:
pivot_table

In [ ]:
sns.heatmap(pivot_table)
plt.show()